In [14]:
import dataclasses
import datetime
import functools
import math
import re
from typing import Optional

import cartopy.crs as ccrs
from google.cloud import storage
from graphcast import autoregressive
from graphcast import casting
from graphcast import checkpoint
from graphcast import data_utils
from graphcast import graphcast
from graphcast import normalization
from graphcast import rollout
from graphcast import xarray_jax
from graphcast import xarray_tree
from IPython.display import HTML
import ipywidgets as widgets
import haiku as hk
import jax
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation
import numpy as np
import xarray
import his_utils

def parse_file_parts(file_name):
  return dict(part.split("-", 1) for part in file_name.split("_"))

gcs_client = storage.Client.create_anonymous_client()
gcs_bucket = gcs_client.get_bucket("dm_graphcast")
gcs_bucket

<Bucket: dm_graphcast>

### Set the model and properties

In [15]:
# initialization 
# TODO: 여기 값 바꾸면 됨 근데 모델 불러서 쓸거면 상관 없ㅅ음
resolution = 0 # 0.25 or 1.0
mesh_size = 1000 # 4~6
latent_size = 100 # 2^4 ~ 2^9
gnn_msg_steps = 10 # 1~32
pressure_levels = 37 # 13, 25, 37
hidden_layers = 1 
radius_query_fraction_edge_length = 0.6 # 1로도 가능
params = None
state = {}

In [16]:
# 모델 생성
model_config = graphcast.ModelConfig(
        resolution=resolution, 
        mesh_size=mesh_size,
        latent_size=latent_size,
        gnn_msg_steps=gnn_msg_steps,
        hidden_layers=hidden_layers,
        radius_query_fraction_edge_length=radius_query_fraction_edge_length    
    )

task_config = graphcast.TaskConfig(
        input_variables=(graphcast.TARGET_SURFACE_VARS + graphcast.TARGET_ATMOSPHERIC_VARS + graphcast.FORCING_VARS +
        graphcast.STATIC_VARS),
        target_variables=graphcast.TARGET_SURFACE_VARS + graphcast.TARGET_ATMOSPHERIC_VARS,
        forcing_variables=graphcast.FORCING_VARS,
        pressure_levels=graphcast.PRESSURE_LEVELS[pressure_levels],
        input_duration="12h"
    )

model_config

ModelConfig(resolution=0, mesh_size=1000, latent_size=100, gnn_msg_steps=10, hidden_layers=1, radius_query_fraction_edge_length=0.6, mesh2grid_edge_normalization_factor=None)

In [17]:
# prerequisits

GC_original = 'GraphCast - ERA5 1979-2017 - resolution 0.25 - pressure levels 37 - mesh 2to6 - precipitation input and output.npz'
GC_operational = 'GraphCast_operational - ERA5-HRES 1979-2021 - resolution 0.25 - pressure levels 13 - mesh 2to6 - precipitation output only.npz'
GC_small = 'GraphCast_small - ERA5 1979-2015 - resolution 1.0 - pressure levels 13 - mesh 2to5 - precipitation input and output.npz'

# TODO: 모델 하나 골라 로드하기
pretrained_model = GC_original

with gcs_bucket.blob(f"params/{pretrained_model}").open("rb") as f:
    ckpt = checkpoint.load(f, graphcast.CheckPoint)

    
params = ckpt.params  # 로드된 체크포인트에서 파라미터 가져오기
state = {}  # 초기 상태는 빈 딕셔너리로 설정

# 체크포인트에서 모델 구성 가져오기
model_config = ckpt.model_config
# 체크포인트에서 작업 구성 가져오기
task_config = ckpt.task_config

# 모델 설명 출력
print("Model description:\n", ckpt.description, "\n")
# 모델 라이선스 출력
print("Model license:\n", ckpt.license, "\n")

print("Model config:\n", model_config, "\n")

print("Task config:\n", task_config)

Model description:
 
GraphCast model at 0.25deg resolution, with 37 pressure levels. This model is
trained on ERA5 data from 1979 to 2017, and can be causally evaluated on 2018
and later years. This model takes as inputs `total_precipitation_6hr`. This was
described in the paper
`GraphCast: Learning skillful medium-range global weather forecasting`
(https://arxiv.org/abs/2212.12794).
 

Model license:
 
The model weights are licensed under the Creative Commons
Attribution-NonCommercial-ShareAlike 4.0 International (CC BY-NC-SA 4.0). You
may obtain a copy of the License at:
https://creativecommons.org/licenses/by-nc-sa/4.0/.
The weights were trained on ERA5 data, see README for attribution statement.
 

Model config:
 ModelConfig(resolution=0.25, mesh_size=6, latent_size=512, gnn_msg_steps=16, hidden_layers=1, radius_query_fraction_edge_length=0.5999912857713345, mesh2grid_edge_normalization_factor=0.6180338738074472) 

Task config:
 TaskConfig(input_variables=('2m_temperature', 'mean_s

### Load Data

In [18]:
# load data

scale =5

file_name = f'testdata/2021-06-26/ERA5_input_perturbed_{scale}std.nc'

dataset = xarray.open_dataset(file_name)
dataset
# print("TOA_solar_incident_radiation" in dataset.data_vars)
# dataset.drop_vars("TOA_solar_incident_radiation") #       <---- TOA 사용 여부 변경!

<xarray.Dataset>
Dimensions:                       (lon: 1440, lat: 721, level: 37, time: 20,
                                   batch: 1)
Coordinates:
  * lon                           (lon) float32 0.0 0.25 0.5 ... 359.5 359.8
  * lat                           (lat) float32 -90.0 -89.75 ... 89.75 90.0
  * level                         (level) int32 1 2 3 5 7 ... 925 950 975 1000
  * time                          (time) timedelta64[ns] 0 days 00:00:00 ... ...
    datetime                      (batch, time) datetime64[ns] ...
  * batch                         (batch) int32 0
Data variables: (12/14)
    geopotential_at_surface       (lat, lon) float32 ...
    land_sea_mask                 (lat, lon) float32 ...
    10m_u_component_of_wind       (batch, time, lat, lon) float32 ...
    10m_v_component_of_wind       (batch, time, lat, lon) float32 ...
    2m_temperature                (batch, time, lat, lon) float64 ...
    mean_sea_level_pressure       (batch, time, lat, lon) float32 ...
    ...                            ...
    geopotential                  (batch, time, level, lat, lon) float32 ...
    specific_humidity             (batch, time, level, lat, lon) float32 ...
    temperature                   (batch, time, level, lat, lon) float32 ...
    u_component_of_wind           (batch, time, level, lat, lon) float32 ...
    v_component_of_wind           (batch, time, level, lat, lon) float32 ...
    vertical_velocity             (batch, time, level, lat, lon) float32 ...
Attributes:
    CDI:          Climate Data Interface version 1.9.3 (http://mpimet.mpg.de/...
    Conventions:  CF-1.6
    history:      Mon Aug 19 11:29:47 2024: cdo -f nc copy adaptor.mars.inter...
    institution:  European Centre for Medium-Range Weather Forecasts
    CDO:          Climate Data Operators version 1.9.3 (http://mpimet.mpg.de/...

In [19]:
# 점검

# 데이터셋의 시간 차원이 최소 3인지 확인 (입력용 2, 목표용 1 이상)
assert dataset.dims["time"] >= 3  # 2 for input, >=1 for targets

# 데이터셋의 시간 크기 확인
total_time_steps = dataset.sizes["time"] - 2

total_time_steps

18

In [20]:
# split dataset
# train_steps = 0
eval_steps = 1

In [21]:
# train_inputs, train_targets, train_forcings = data_utils.extract_inputs_targets_forcings(
#     dataset, 
#     target_lead_times=slice("6h", f"{train_steps * 6}h"), 
#     **dataclasses.asdict(task_config)
# )

eval_inputs, _, _ = data_utils.extract_inputs_targets_forcings(
    dataset, 
    target_lead_times=slice("6h", f"{eval_steps * 6}h"), 
    **dataclasses.asdict(task_config)
)

|| data_utils.py -> add_derived_vars() init. ||


# 실행

데이터 뽑기는 여기 앞까지

In [22]:
# Print the dimensions of the example batch and extract|ed training and evaluation data
print("All Examples:  ", dataset.dims.mapping)
# print("----------------------------------------------")
# print("Train Inputs:  ", train_inputs.dims.mapping)
# print("Train Targets: ", train_targets.dims.mapping)
# print("Train Forcings:", train_forcings.dims.mapping)
print("----------------------------------------------")
print("Eval Inputs:   ", eval_inputs.dims.mapping)
# print("Eval Targets:  ", eval_targets.dims.mapping)
# print("Eval Forcings: ", eval_forcings.dims.mapping)
# print("----------------------------------------------")
# print("Eval Inputs:   \n", eval_inputs)
# print("Eval Targets:  \n", eval_targets)
# print("Eval Forcings: \n", eval_forcings)

All Examples:   {'lon': 1440, 'lat': 721, 'level': 37, 'time': 20, 'batch': 1}
----------------------------------------------
Eval Inputs:    {'batch': 1, 'time': 2, 'lat': 721, 'lon': 1440, 'level': 37}


In [23]:
# @title Load normalization data

diffs_stddev_by_level = xarray.open_dataset("testdata/stats/stats_diffs_stddev_by_level.nc")

mean_by_level = xarray.open_dataset("testdata/stats/stats_mean_by_level.nc")

stddev_by_level = xarray.open_dataset("testdata/stats/stats_stddev_by_level.nc")


eval_steps = 40

target_template = his_utils.create_target_dataset(time_steps=eval_steps, 
                   resolution=model_config.resolution, 
                   pressure_levels=len(task_config.pressure_levels))

# start_time: dataset.datetime[0]
forcings = his_utils.create_forcing_dataset(time_steps=eval_steps,
                                              resolution=model_config.resolution,
                                              start_time=dataset.datetime.values[0, 0])

|| data_utils.py -> add_derived_vars() init. ||


In [24]:
# JIT 컴파일된 함수를 생성하고, 필요한 경우 랜덤 가중치를 초기화합니다

def construct_wrapped_graphcast(
    model_config: graphcast.ModelConfig,
    task_config: graphcast.TaskConfig):
  """GraphCast 예측기를 구성하고 래핑합니다."""
  predictor = graphcast.GraphCast(model_config, task_config)
  predictor = casting.Bfloat16Cast(predictor)
  predictor = normalization.InputsAndResiduals(
      predictor,
      diffs_stddev_by_level=diffs_stddev_by_level,
      mean_by_level=mean_by_level,
      stddev_by_level=stddev_by_level)
  predictor = autoregressive.Predictor(predictor, gradient_checkpointing=True)
  return predictor

@hk.transform_with_state
def run_forward(model_config, task_config, inputs, targets_template, forcings):
    """구성된 예측기를 사용하여 순전파를 실행합니다."""
    predictor = construct_wrapped_graphcast(model_config, task_config)
    return predictor(inputs, targets_template=targets_template, forcings=forcings)

@hk.transform_with_state
def loss_fn(model_config, task_config, inputs, targets, forcings):
    """구성된 예측기를 사용하여 손실과 진단 정보를 계산합니다."""
    predictor = construct_wrapped_graphcast(model_config, task_config)
    loss, diagnostics = predictor.loss(inputs, targets, forcings)
    return xarray_tree.map_structure(
        lambda x: xarray_jax.unwrap_data(x.mean(), require_jax=True),
        (loss, diagnostics)
    )

def grads_fn(params, state, model_config, task_config, inputs, targets, forcings):
  """매개변수에 대한 손실 함수의 그래디언트를 계산합니다."""
  def _aux(params, state, i, t, f):
    (loss, diagnostics), next_state = loss_fn.apply(
        params, state, jax.random.PRNGKey(0), model_config, task_config,
        i, t, f)
    return loss, (diagnostics, next_state)
  (loss, (diagnostics, next_state)), grads = jax.value_and_grad(
      _aux, has_aux=True)(params, state, inputs, targets, forcings)
  return loss, diagnostics, next_state, grads

def with_configs(fn):
  """모델 및 작업 구성을 적용하는 유틸리티 함수입니다."""
  return functools.partial(
      fn, model_config=model_config, task_config=task_config)

def with_params(fn):
  """함수가 항상 매개변수와 상태를 받도록 보장합니다."""
  return functools.partial(fn, params=params, state=state)

def drop_state(fn):
  """예측만 반환합니다. 롤아웃 코드에 필요합니다."""
  return lambda **kw: fn(**kw)[0]

init_jitted = jax.jit(with_configs(run_forward.init))

if params is None:
    # 매개변수와 상태가 아직 설정되지 않은 경우 초기화합니다.
    params, state = init_jitted(
        rng=jax.random.PRNGKey(0),
        inputs=train_inputs,
        targets_template=train_targets,
        forcings=train_forcings
    )

# 필요한 구성과 매개변수로 함수들을 JIT 컴파일합니다.
loss_fn_jitted = drop_state(with_params(jax.jit(with_configs(loss_fn.apply))))
grads_fn_jitted = with_params(jax.jit(with_configs(grads_fn)))
run_forward_jitted = drop_state(with_params(jax.jit(with_configs(run_forward.apply))))

# Ensure that the model resolution matches the data resolution
assert model_config.resolution in (0, 360. / eval_inputs.sizes["lon"]), (
    "Model resolution doesn't match the data resolution. You likely want to "
    "re-filter the dataset list, and download the correct data.")

In [25]:
print(jax.__version__)
print(jax.devices())

# Perform autoregressive rollout to generate predictions
predictions = rollout.chunked_prediction(
    run_forward_jitted,
    rng=jax.random.PRNGKey(0),
    inputs=eval_inputs,
    targets_template=target_template,
    forcings=forcings
)

# Display the predictions
predictions

0.4.23
[cuda(id=0), cuda(id=1)]
|| rollout.py -> chunked_prediction() init. ||
|| rollout.py -> chunked_prediction_generator() init. ||


|| graphcast.py -> __call__() init ||
|| graphcast.py -> _maybe_init() init ||
|| graphcast.py -> _maybe_init() done ||
|| graphcast.py -> _inputs_to_grid_node_features() init ||
|| graphcast.py -> _inputs_to_grid_node_features() done ||
|| graphcast.py -> _run_grid2mesh_gnn() init ||
|| graphcast.py -> _run_grid2mesh_gnn() done ||
latent_mesh_nodes shape: (40962, 1, 512)
|| graphcast.py -> _run_mesh_gnn() init ||
|| graphcast.py -> _run_mesh_gnn() done ||
updated_latent_mesh_nodes shape: (40962, 1, 512)
output_grid_nodes shape: (1038240, 1, 227)
|| graphcast.py -> __call__() done ||


2024-09-12 08:01:40.368062: E external/xla/xla/service/slow_operation_alarm.cc:65] Constant folding an instruction is taking > 1s:

  %pad.1 = bf16[40962,1,474]{2,1,0} pad(bf16[40962,1,3]{2,1,0} %constant.348, bf16[] %constant.349), padding=0_0x0_0x471_0, metadata={op_name="jit(<unnamed wrapped function>)/jit(main)/while/body/concatenate[dimension=2]" source_file="/home/hiskim1/graphcast/graphcast/graphcast.py" source_line=645}

This isn't necessarily a bug; constant-folding is inherently a trade-off between compilation time and speed at runtime. XLA has some guards that attempt to keep constant folding from taking too long, but fundamentally you'll always be able to come up with an input program that takes a long time.

If you'd like to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
2024-09-12 08:01:41.761323: E external/xla/xla/service/slow_operation_alarm.cc:133] The operation took 2.393834838s
Constant folding an instruction is taking > 1s:

  

|| rollout.py -> chunked_prediction_generator() done ||
|| rollout.py -> chunked_prediction_generator() done ||
|| rollout.py -> chunked_prediction_generator() done ||
|| rollout.py -> chunked_prediction_generator() done ||
|| rollout.py -> chunked_prediction_generator() done ||
|| rollout.py -> chunked_prediction_generator() done ||
|| rollout.py -> chunked_prediction_generator() done ||
|| rollout.py -> chunked_prediction_generator() done ||
|| rollout.py -> chunked_prediction_generator() done ||
|| rollout.py -> chunked_prediction_generator() done ||
|| rollout.py -> chunked_prediction_generator() done ||
|| rollout.py -> chunked_prediction_generator() done ||
|| rollout.py -> chunked_prediction_generator() done ||
|| rollout.py -> chunked_prediction_generator() done ||
|| rollout.py -> chunked_prediction_generator() done ||
|| rollout.py -> chunked_prediction_generator() done ||
|| rollout.py -> chunked_prediction_generator() done ||
|| rollout.py -> chunked_prediction_generator() 

<xarray.Dataset>
Dimensions:                  (time: 40, batch: 1, lat: 721, lon: 1440, level: 37)
Coordinates:
  * time                     (time) timedelta64[ns] 0 days 06:00:00 ... 10 da...
  * lon                      (lon) float32 0.0 0.25 0.5 ... 359.2 359.5 359.8
  * lat                      (lat) float32 -90.0 -89.75 -89.5 ... 89.75 90.0
  * level                    (level) int32 1 2 3 5 7 10 ... 900 925 950 975 1000
Dimensions without coordinates: batch
Data variables:
    10m_u_component_of_wind  (time, batch, lat, lon) float32 -0.08371 ... 2.482
    10m_v_component_of_wind  (time, batch, lat, lon) float32 -0.9205 ... -0.8352
    2m_temperature           (time, batch, lat, lon) float32 220.4 ... 274.1
    geopotential             (time, batch, lat, lon, level) float32 4.014e+05...
    mean_sea_level_pressure  (time, batch, lat, lon) float32 1e+05 ... 1.014e+05
    specific_humidity        (time, batch, lat, lon, level) float32 3.48e-06 ...
    temperature              (time, batch, lat, lon, level) float32 237.9 ......
    total_precipitation_6hr  (time, batch, lat, lon) float32 0.0001031 ... 0....
    u_component_of_wind      (time, batch, lat, lon, level) float32 2.502 ......
    v_component_of_wind      (time, batch, lat, lon, level) float32 0.3833 .....
    vertical_velocity        (time, batch, lat, lon, level) float32 -0.000851...

In [26]:
predictions.to_netcdf(f"testdata/2021-06-26/prediction_40step_{scale}std_perturb.nc")

# Plot

이쁘게 포장하는 공정은 여기부터

In [ ]:
import xarray as xr
import numpy as np
import his_utils, his_plot
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from matplotlib.colors import TwoSlopeNorm
from multiprocessing import Pool

target_var_list = [
 '2m_temperature']

dataset = xr.open_dataset("testdata/stats/40yr_std_daily.nc")
dataset = his_utils.convert_scale(dataset)
dataset["2m_temperature"].dims

In [9]:

for var in target_var_list:
    weights = np.cos(np.deg2rad(dataset.lat))
    weights.name = "weights"
    weighted = dataset[var].weighted(weights)
    mean_val = dataset[var].mean(('lat', 'lon')).max()
    std_val = dataset[var].std(('lat', 'lon')).max()

    norm = TwoSlopeNorm(vcenter=mean_val)

    title = "std"

    arg_list = [(dataset[var].isel(hour=time_index), 
                 var, 
                 "platecarree", 
                 "RdBu_r", 
                 f'{var} {title} at t:{time_index}', 
                 f'figure/std {var}_{time_index}.png',
                 norm)
                   for time_index in range(len(dataset.hour))]
        
    with Pool() as pool:
        pool.map(his_plot.plot, arg_list)

## 움짤 만들기

In [ ]:
from PIL import Image
from pathlib import Path
import re
import os
import his_utils

# 'figure' 디렉토리의 경로를 지정합니다.
figure_dir = 'figure'

# # 파일 이름에서 날짜와 시간을 추출하는 정규표현식
# pattern = r'polar_GC_temperature_(\d{4}-\d{2}-\d{2}T\d{2}:\d{2}).png'

# # 파일 이름에서 날짜와 시간을 추출하여 리스트를 만듭니다.
# date_list = sorted([re.search(pattern, f).group(1) for f in os.listdir(figure_dir) if f.startswith('polar_GC_temperature_') and f.endswith('.png')])

# 정렬된 date_list를 사용하여 image_frames를 생성합니다.
image_frames = [Image.open(os.path.join(figure_dir, f'total_precipitation_6hrRR_{date}.png')) for date in range(0,14)]

# GIF를 저장합니다.
his_utils.save_gif(image_frames, 'tp_6hr diff.gif', duration=700)

In [ ]:
from multiprocessing import Pool
from PIL import Image
from pathlib import Path
import os
import his_plot

surface_list = [
    '10m_u_component_of_wind',
    '10m_v_component_of_wind',
    '2m_temperature',
    'mean_sea_level_pressure',
    'total_precipitation_6hr'
]

pressure_list = [
    'geopotential',
    'specific_humidity',
    'temperature',
    'u_component_of_wind',
    'v_component_of_wind',
    'vertical_velocity',
]

title = "ERA5 prediction: 1st - 2nd"

def process_var(args):
    var, duration, level = args
    image_frames = []
    for time_index in range(8):
        if level is not None:
            filename = f'{title} {var}_{time_index}_{level}.png'
        else:
            filename = f'{title} {var}_{time_index}.png'
        
        file_path = Path("figure") / filename
        if file_path.exists():
            with Image.open(file_path) as img:
                image_frames.append(img.copy())
        else:
            print(f"Warning: File not found - {file_path}")
    
    if image_frames:
        output_filename = f'2022-01-01_{var}.gif'
        if level is not None:
            output_filename = f'2022-01-01_{var}_{level}.gif'
        his_plot.save_gif(image_frames, output_filename, duration=duration)
    else:
        print(f"No images found for variable: {var}")

duration = 700
with Pool() as pool:
    # Process surface variables
    surface_args = [(var, duration, None) for var in surface_list]
    pool.map(process_var, surface_args)
    
    # Process pressure variables
    pressure_args = [(var, duration, level) 
                     for level in range(37) 
                     for var in pressure_list]
    pool.map(process_var, pressure_args)


# 연습장


In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import xarray as xr
import numpy as np
from matplotlib.colors import TwoSlopeNorm
import matplotlib.pyplot as plt
from multiprocessing import Pool

ERA5 = xr.open_dataset(f'testdata/2022-01-01RR.nc')
google = xr.open_dataset('testdata/source-era5_date-2022-01-01_res-0.25_levels-37_steps-12.nc').drop_vars('toa_incident_solar_radiation')

def plot(args):
    dataset, target_var, time_index = args

    fig = plt.figure(figsize=(20, 10))
    ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=180))

    data = dataset * 1000

    im = ax.pcolormesh(data.lon, data.lat, data.isel(time=time_index).squeeze(), 
                   transform=ccrs.PlateCarree(), 
                   cmap='Blues',
                   norm=TwoSlopeNorm(vmin=0, vcenter=0.5, vmax=20),
                   shading='auto')
    
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS)
    ax.gridlines(draw_labels=True)

    flag = 'batch' in dataset.coords

    plt.title(f'{flag} {target_var}\nTime: {time_index}')
    ax.set_global()

    plt.savefig(f'figure/{flag} {target_var}_{time_index}.png', dpi=300, bbox_inches='tight')
    plt.close()


with Pool() as pool:
    args_list = [(ERA5['total_precipitation_6hr'], 'total_precipitation_6hr',time_index) for time_index in range(0, 14)]
    pool.map(plot, args_list)

with Pool() as pool:
    args_list = [(google['total_precipitation_6hr'], 'total_precipitation_6hr', time_index) for time_index in range(0, 14)]
    pool.map(plot, args_list)

In [ ]:
import xarray as xr

google_mean = xr.open_dataset("testdata/stats_mean_by_level.nc")
google_mean.data_vars
VAR_LIST = [
    "10m_u_component_of_wind",
    "10m_v_component_of_wind",
    "2m_temperature",
    "mean_sea_level_pressure",    
    "geopotential",
    "specific_humidity",
    "temperature",
    "u_component_of_wind",
    "v_component_of_wind",
    "vertical_velocity",
]

# for var in VAR_LIST:
#     print(var)
#     print(google_mean[var].values)

google_mean

In [ ]:
dataset_1 = xarray.open_dataset("testdata/2021-06-26/prediction_40step_no_perturb.nc")
# dataset_2 = xarray.open_dataset("predictions_ERA5_2022-01-01T00h_20step_2nd.nc")

dataset_1["2m_temperature"]

# def calculate_error(arg):
#     ds_1, ds_2, var = arg
#     mse = mean_squared_error(ds_1, ds_2)

#     print(f'{var} mse: {mean_val}')